# Notebook 01 - Pré-processamento de Dados (MG, RJ, SP)

Este notebook tem como objetivo carregar os arquivos CSV originais, aplicar um pré-processamento inicial e gerar versões mais leves para análises futuras.

---

## 1. Importação de bibliotecas


In [52]:
#instalação, se não tiver.
%pip install pandas
%pip install fastparquet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [53]:
import pandas as pd
import os

## 2. Definição de caminhos dos arquivos

Os dados originais estão na pasta `data/`.  
Os arquivos reduzidos serão salvos em `data/processed/`.

In [54]:
# Dicionário com os arquivos
files = {
    "MG": "../data/sgmg.csv",
    "RJ": "../data/sgrj.csv",
    "SP": "../data/sgsp.csv"
}

## 3. Contagem de Linhas não padronizadas

In [55]:
def contar_bad_lines(path, expected_fields=64, sep=";",engine="python"):
    bad_count = 0
    with open(path, "r", encoding="utf-8") as f:
        for i, line in enumerate(f):
            if line.count(sep) != expected_fields - 1:  # separadores = campos - 1
                bad_count += 1
    return bad_count

In [56]:
#Conta as linhas quebradas (bad lines = número errado de colunas)
for uf, path in files.items():
    if os.path.exists(path):
        print(f"\n--- {uf} ---")
        
        # 1. Contar linhas ruins
        bad_count = contar_bad_lines(path, expected_fields=64, sep=";")
        print(f"Linhas problemáticas detectadas: {bad_count}")


--- MG ---
Linhas problemáticas detectadas: 611

--- RJ ---
Linhas problemáticas detectadas: 379

--- SP ---
Linhas problemáticas detectadas: 2251


##  4. Carregar apenas as linhas consistentes
Verifica-se também informações básicas para check de consistência

In [60]:
for uf, path in files.items():
    if os.path.exists(path):
        print(f"\n--- {uf} ---")
        # Carregar somente as primeiras linhas para testar
        df = pd.read_csv(path, sep=";", low_memory=False, on_bad_lines="skip")
        
        # Padronizar nomes de colunas
        df.columns = df.columns.str.lower()
        print("Colunas:", df.columns.tolist(), "...")

        
        if "municipio" in df.columns:
            print("Shape:", df.shape)
            
            # Amostra de municípios únicos
            print("\nMunicípios (amostra):")
            print(df["municipio"].dropna().unique()[:20])
            
            # Frequência dos municípios mais comuns
            print("\nTop 10 municípios:")
            print(df["municipio"].value_counts().head(10))
        else:
            print("Coluna 'municipio' não encontrada nesse arquivo.")
    else:
        print(f"Arquivo não encontrado: {path}")
     # salvar em Parquet (mais leve que CSV)
    out_path = f"../data/processed/{uf.lower()}_clean.parquet"
   
    df.to_parquet(out_path, index=False, engine="fastparquet", compression="gzip")
    print(f"Salvo em {out_path} com fastparquet e compressão gzip.")



--- MG ---
Colunas: ['sintomas', 'profissionalsaude', 'racacor', 'outrossintomas', 'outrascondicoes', 'profissionalseguranca', 'cbo', 'condicoes', 'sexo', 'estado', 'estadoibge', 'municipio', 'municipioibge', 'origem', 'estadonotificacao', 'municipionotificacao', 'municipionotificacaoibge', 'evolucaocaso', 'classificacaofinal', 'codigoestrategiacovid', 'codigobuscaativaassintomatico', 'outrobuscaativaassintomatico', 'codigotriagempopulacaoespecifica', 'outrotriagempopulacaoespecifica', 'codigolocalrealizacaotestagem', 'outrolocalrealizacaotestagem', 'codigorecebeuvacina', 'codigolaboratorioprimeiradose', 'codigolaboratoriosegundadose', 'loteprimeiradose', 'lotesegundadose', 'codigocontemcomunidadetradicional', 'source_id', 'excluido', 'validado', 'codigodosesvacina', 'estadonotificacaoibge', 'totaltestesrealizados', 'datanotificacao', 'datainiciosintomas', 'dataencerramento', 'dataprimeiradose', 'datasegundadose', 'codigoestadoteste1', 'codigotipoteste1', 'codigofabricanteteste1', 'co